# Music
<img src="Music/Y4_Large_reduc.jpg" align="right" width="64" height="64" />

Zen's Music Playround

# Introduction

This page contains some experiments with Jupyter and playing with my music collection. To play happily, we have divided this page into multiple sections.

These sections are:
* [Setup](#Setup "Load the Music Library class and variables")
* [Get random albums](#Random-Albums "Get 5 random albums")
* [Get random album and cover](#Random-Album-and-Cover "Get a random album and it's track listing.")

For useful help, you can check out:
* The [Jupyter Widget List](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)


## Setup

Execute the cell below to instantiate the MusicLib class and make it available as *lib*.

In [15]:
from os import listdir
from os.path import join, isdir, exists, basename
from random import sample
from glob import glob
import ipywidgets as widgets


class MusicLib(object):
    """
    Class for fetching information about our music library, as an experiment
    in jupyter
    """
   
    def __init__(self, path):
       self.path = path 
    
    def artists(self):
        """ Return a list of lartists. """
        return [name for name in listdir(self.path) if isdir(join(self.path, name))]

    def get_random_artists(self, number):
        """ Return a random list of *number* artists. """
        artists = self.artists()
        return sample(artists, number)

    def get_random_albums(self, artist, number):
        """ Return a random list of *number* albums by *artist*. """
        albums = self.albums(artist)
        return sample(albums, number)

    def albums(self, artist):
        """ Return a list of albums for the *artist*. """
        path = join(self.path, artist)
        return [name for name in listdir(path) if isdir(join(path, name))]
    
    def get_album_cover(self, artist, album, size=300):
        """ Return the album cover for the specified album. """
        path = join(self.path, artist, album)
        pattern = "cover.*"
        matches = glob(join(path, pattern))
        if matches:
            fname = matches[0]
            with open(fname, "rb") as f:
                return widgets.Image(
                    value=f.read(),
                    format=fname[:-3],
                    width=size,
                    height=size,
                )
    def get_album_tracks(self, artist, album):
        """ Return a list of the album tracks"""
        def get_name(fname):
            """"Return the nice, cleaned name of the track"""
            return basename(fname)  # [:-4]
        
        path = join(self.path, artist, album)
        pattern = "*.mp3"
        matches = glob(join(path, pattern))        
        return sorted([get_name(f) for f in matches])

    def get_album_summary(self, artist, album):
        """ Return a pretty album summary """
        cover = self.get_album_cover(artist, album)
        items = [widgets.Label("{0}: {1}".format(artist, album))]
        [items.append(widgets.Label(track, width=200)) for track in self.get_album_tracks(artist,
                                                                album)]
        right = widgets.VBox(items, width=300)
        return widgets.HBox([cover, right])
    
library = "/home/fruitbat/Music/"
lib = MusicLib(library)           

[Back to top](#Introduction "Back the top of the page")

## Random Albums

In [14]:
artists = lib.get_random_artists(5)
for artist in artists:
    albums = lib.albums(artist)
    print("{0}: {1} albums".format(artist, len(albums)))
    [print("    {0}".format(album)) for album in albums]

Gary Moore: 1 albums
    Back On The Streets
Infected Mushroom: 9 albums
    Return to the Sauce
    I'm The Supervisor
    Vicious Delicious
    Converting Vegetarians - The Other Side
    Legend Of The Black Shawarma
    Classical Mushroom
    B.P. Empire
    Army of Mushrooms
    Converting Vegetarians - Trance Side
George Michael: 1 albums
    Older
Def Leppard: 2 albums
    Retro Active
    Vault
Beck: 1 albums
    Odelay!


[Back to top](#Introduction "Back the top of the page")

## Random Album and Cover

In [13]:
artist =  lib.get_random_artists(1)[0]
album =  lib.get_random_albums(artist, 1)[0]
print("{0}: {1}".format(artist, album))
[print(track) for track in lib.get_album_tracks(artist, album)]
lib.get_album_cover(artist, album, 200)


James: Laid
01 - Out To Get You.mp3
02 - Sometimes (Lester Piggott).mp3
03 - Dream Thrum.mp3
04 - One Of The Three.mp3
05 - Say Something.mp3
06 - Five-O.mp3
07 - P.S..mp3
08 - Everybody Knows.mp3
09 - Knuckle Too Far.mp3
10 - Low Low Low.mp3
11 - Laid.mp3
12 - Lullaby.mp3
13 - Skindiving.mp3


[Back to top](#Introduction "Back the top of the page")